### _I, Library:_

In [88]:
import pandas as pd
import re
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import pytz

### _II, Scraping data:_

- The data is about the real estate for sale in my city, Hanoi

In [89]:
url = "https://alonhadat.com.vn/nha-dat/can-ban/dat-tho-cu-dat-o/1/ha-noi.html"

#url = "https://alonhadat.com.vn/nha-dat/can-ban/nha-dat/1/ha-noi/trang--2.html"
#url = "https://alonhadat.com.vn/nha-dat/can-ban/nha-dat/1/ha-noi.html"
def scraping(url):
    """
    Scraping data.
    """
    result = requests.get(url)
    c = result.content
    soup = BeautifulSoup(c,"html.parser")
    lists = soup.find_all("div" , class_ = "content-item")
    df_list = []
    for e in lists:

        try:
            date = e.find("div", class_ = "ct_date").text
            if date == "Hôm nay":
                tz_VN = pytz.timezone('Asia/Bangkok')
                today = datetime.now(tz_VN)
                if today.month <=10:
                    date = str(today.day) + "/0" + str(today.month) + "/" + str(today.year)
                else:
                    date = str(today.day) + "/" + str(today.month) + "/" + str(today.year)
            elif date == "Hôm qua":
                tz_VN = pytz.timezone('Asia/Bangkok')
                today = datetime.now(tz_VN)
                if today.month <=10:
                    date = str(today.day-1) + "/0" + str(today.month) + "/" + str(today.year)
                else:
                    date = str(today.day-1) + "/" + str(today.month) + "/" + str(today.year)
        except:
            date = '-'
            
            
        try:
            road_width = e.find("span", class_ = "road-width").text
        except:
            road_width = "-"
        try:
            area = e.find("div", class_ = "ct_dt").text.split(": ")[1]
        except:
            area = "-"

        try:
            size = e.find("div", class_ = "ct_kt").text.split(": ")[1]
        except:
            size = "-"

        try:
            direction = e.find("div", class_ = "ct_direct").text.split(": ")[1]
        except:
            direction = "-"
        try:
            price = e.find("div", class_ = "ct_price").text.split(": ")[1]
        except:
            price = "-"
            
        try:
            location = e.find("div", class_  = "ct_dis").text
        except:
            price = "-"
        combine = [ date, road_width, area, size, direction, price, location]
        df_list.append(combine)
    
    return df_list
first_page = scraping(url)
first_page

[['27/08/2022',
  '7m',
  '199 m2',
  '---',
  'Đông Bắc',
  '62 tỷ ',
  'Đường Quảng An, Phường Quảng An, Quận Tây Hồ, Hà Nội'],
 ['27/08/2022',
  '24m',
  '300 m2',
  '10x30m',
  '_',
  '1,1 tỷ ',
  'Đường Bình Minh, Xã Đặng Xá, Huyện Gia Lâm, Hà Nội'],
 ['27/08/2022',
  '5m',
  '780 m2',
  '---',
  '_',
  '2 tỷ ',
  'Xã Khánh Thượng, Huyện Ba Vì, Hà Nội'],
 ['27/08/2022',
  '6m',
  '168 m2',
  '---',
  '_',
  '45,8 tỷ ',
  'Phố Nhật Chiêu, Phường Xuân La, Quận Tây Hồ, Hà Nội'],
 ['27/08/2022',
  '3m',
  '40 m2',
  '4,5x9m',
  '_',
  '5,3 tỷ ',
  'Phố Trần Thái Tông, Phường Dịch Vọng, Quận Cầu Giấy, Hà Nội'],
 ['26/08/2022',
  '-',
  '58 m2',
  '---',
  '_',
  '6,7 tỷ ',
  'Đường Linh Đường, Phường Hoàng Liệt, Quận Hoàng Mai, Hà Nội'],
 ['26/08/2022',
  '6m',
  '65 m2',
  '5x14m',
  'Đông Nam',
  '1 tỷ ',
  'Tỉnh lộ 420, Xã Bình Yên, Huyện Thạch Thất, Hà Nội'],
 ['23/08/2022',
  '3m',
  '62 m2',
  '6x9m',
  '_',
  '880 triệu ',
  'Đường Tỉnh Lộ 131, Xã Hiền Ninh, Huyện Sóc Sơn, Hà Nộ

In [90]:
final_list = []
for numb in range(1,2000):
    try:
        url1 = "https://alonhadat.com.vn/nha-dat/can-ban/dat-tho-cu-dat-o/1/ha-noi/trang--"+str(numb)+".html"
        new = scraping(url1)
        final_list+=new
    except:
        break

df = pd.DataFrame(final_list, columns=['Date', 'Road_width(m)', 'Area(m2)', 'Size(m)', 'Direction', 'Price', 'Location'])
#df.shape
df

,Date,Road_width(m),Area(m2),Size(m),Direction,Price,Location
0,27/08/2022,7m,199 m2,---,Đông Bắc,62 tỷ,"Đường Quảng An, Phường Quảng An, Quận Tây Hồ, ..."
1,27/08/2022,24m,300 m2,10x30m,_,"1,1 tỷ","Đường Bình Minh, Xã Đặng Xá, Huyện Gia Lâm, Hà..."
2,27/08/2022,5m,780 m2,---,_,2 tỷ,"Xã Khánh Thượng, Huyện Ba Vì, Hà Nội"
3,27/08/2022,6m,168 m2,---,_,"45,8 tỷ","Phố Nhật Chiêu, Phường Xuân La, Quận Tây Hồ, H..."
4,27/08/2022,3m,40 m2,"4,5x9m",_,"5,3 tỷ","Phố Trần Thái Tông, Phường Dịch Vọng, Quận Cầu..."
...,...,...,...,...,...,...,...
23753,19/06/2022,6m,82 m2,4x21m,Đông Nam,"7,2 tỷ","Đường Lai Xá, Xã Kim Chung, Huyện Hoài Đức, Hà..."
23754,18/06/2022,6m,82 m2,4x21m,Đông Nam,"7,2 tỷ","Đường Lai Xá, Xã Kim Chung, Huyện Hoài Đức, Hà..."
23755,18/06/2022,4m,1.800 m2,20x9m,_,"9,9 tỷ","Quốc lộ 6, Xã Đông Sơn, Huyện Chương Mỹ, Hà Nội"
23756,17/06/2022,5m,75 m2,---,_,"2,27 tỷ","Xã Vạn Phúc, Huyện Thanh Trì, Hà Nội"


In [91]:


df.to_excel("hnrealestate.xlsx", sheet_name='Sheet1',index=False)
df.to_csv('realestate.csv', encoding='utf-8')
a = pd.read_excel("hnrealestate.xlsx")
a

,Date,Road_width(m),Area(m2),Size(m),Direction,Price,Location
0,27/08/2022,7m,199 m2,---,Đông Bắc,62 tỷ,"Đường Quảng An, Phường Quảng An, Quận Tây Hồ, ..."
1,27/08/2022,24m,300 m2,10x30m,_,"1,1 tỷ","Đường Bình Minh, Xã Đặng Xá, Huyện Gia Lâm, Hà..."
2,27/08/2022,5m,780 m2,---,_,2 tỷ,"Xã Khánh Thượng, Huyện Ba Vì, Hà Nội"
3,27/08/2022,6m,168 m2,---,_,"45,8 tỷ","Phố Nhật Chiêu, Phường Xuân La, Quận Tây Hồ, H..."
4,27/08/2022,3m,40 m2,"4,5x9m",_,"5,3 tỷ","Phố Trần Thái Tông, Phường Dịch Vọng, Quận Cầu..."
...,...,...,...,...,...,...,...
23753,19/06/2022,6m,82 m2,4x21m,Đông Nam,"7,2 tỷ","Đường Lai Xá, Xã Kim Chung, Huyện Hoài Đức, Hà..."
23754,18/06/2022,6m,82 m2,4x21m,Đông Nam,"7,2 tỷ","Đường Lai Xá, Xã Kim Chung, Huyện Hoài Đức, Hà..."
23755,18/06/2022,4m,1.800 m2,20x9m,_,"9,9 tỷ","Quốc lộ 6, Xã Đông Sơn, Huyện Chương Mỹ, Hà Nội"
23756,17/06/2022,5m,75 m2,---,_,"2,27 tỷ","Xã Vạn Phúc, Huyện Thanh Trì, Hà Nội"


In [93]:
def compare_time(date):
    """
    return true if time value that further than the latest day of dateset, otherwise, false
    """
    newdate1 = datetime.strptime(date, "%d/%m/%Y")

    max_date = pd.to_datetime(df["Date"],format = '%d/%m/%Y').max()
    max_time = max_date + timedelta(seconds=59, minutes=59, hours=23)
    if newdate1 >= max_time:
        return True
    else:
        return False

In [94]:
def convert_time(date):
    """
    Convert the date to str format 
    """
    if date == "Hôm nay":
        tz_VN = pytz.timezone('Asia/Bangkok')
        today = datetime.now(tz_VN)
        if today.month <=10:
            date = str(today.day) + "/0" + str(today.month) + "/" + str(today.year)
        else:
            date = str(today.day) + "/" + str(today.month) + "/" + str(today.year)
    elif date == "Hôm qua":
        tz_VN = pytz.timezone('Asia/Bangkok')
        today = datetime.now(tz_VN)
        if today.month <=10:
            date = str(today.day-1) + "/0" + str(today.month) + "/" + str(today.year)
        else:
            date = str(today.day-1) + "/" + str(today.month) + "/" + str(today.year)
    return date

In [95]:
url = "https://alonhadat.com.vn/nha-dat/can-ban/dat-tho-cu-dat-o/1/ha-noi.html"
def newday_scraping(url):
    """
    Scraping data.
    """
    result = requests.get(url)
    c = result.content
    soup = BeautifulSoup(c,"html.parser")
    lists = soup.find_all("div" , class_ = "content-item")
    df_list = []
    for e in lists:

        try:
            date = e.find("div", class_ = "ct_date").text
            date = convert_time(date)
            if compare_time(date) == False:
                continue
            else:
                print(date + ":Replicate data")
        except:
            date = '-'
            
            
        try:
            road_width = e.find("span", class_ = "road-width").text
        except:
            road_width = "-"
        try:
            area = e.find("div", class_ = "ct_dt").text.split(": ")[1]
        except:
            area = "-"

        try:
            size = e.find("div", class_ = "ct_kt").text.split(": ")[1]
        except:
            size = "-"

        try:
            direction = e.find("div", class_ = "ct_direct").text.split(": ")[1]
        except:
            direction = "-"
        try:
            price = e.find("div", class_ = "ct_price").text.split(": ")[1]
        except:
            price = "-"
            
        try:
            location = e.find("div", class_  = "ct_dis").text
        except:
            price = "-"
        #print(compare_time(date))
        #print(date)
        combine = [ date, road_width, area, size, direction, price, location]
        df_list.append(combine)
    
    return df_list
first_page = newday_scraping(url)
first_page

[]